# Q-Ellipsoid Method of Feasibility

The point of this code is to find a feasible point of a polygon $P = \{x \in \mathbb{R}^n: Ax \geq b\}$ where $A$ is an $m \times n$ matrix and $b$ is a vector of length $m$. We can find a point $x^*$ such that it violates each constraint by at most $\epsilon$

In [122]:
from qiskit import *
from qiskit.circuit.library import StatePreparation
from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.library import HamiltonianGate
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
simulator = AerSimulator()

import numpy as np
from numpy import linalg
import math
import scipy.linalg as la

from sympy import Matrix
from sympy.physics.quantum import TensorProduct

import matplotlib.pyplot as plt

## Misc. Functions

In [123]:
# Gives the vertical transpose of a horizontal vector. Used in finding new center and ellipse.
def trans(row):
    new= []
    for element in range(len(row)):
        new.append([row[element]])
        
    return np.array(new)

In [124]:
def create_unitary(v):
    return np.hstack((v,la.null_space(v.T)))

In [125]:
# Graphs a 2D ellipse of the form E(c,D)
def graphEllipse(D,c):
    x = np.linspace(-10,10, 1000)
    y = np.linspace(-10,10, 1000)
    x, y = np.meshgrid(x, y)
    
    inD = np.linalg.inv(D)
    
    plt.contour(x,y,((inD[0][0]*(x-c[0])**2) + ((inD[0][1]+inD[1][0])*(y-c[1])*(x-c[0])) + (inD[1][1]*(y-c[1])**2)), [1], colors='blue')
    plt.plot(c[0],c[1], marker='o', color='red')
    plt.show()

### Input and Conditioning

In [127]:
def conditionMatrixVector(A,b):
    A_new = []
    b_new = b
    
    # Get the matrix A to be of the proper dimension so its rows can be encoded as quantum states
    # This also forses there the be zeros in the last column
    # This is so we can initialize the center to have a 1 in the last column so it will always be a unit vector
    for row in range(len(A)):
        new_row = list(A[row])
        
        for zero in range(dim_dif):
            new_row.append(0.0)
        
        A_new.append(new_row)
        
    A_new = np.array(A_new)
    
    # Here, we normalize the rows of A so that they can be encoded as quantum states
    # We scale the entries of b so that the polytope defined remains the same
    for index in range(len(A)):
        b_new[index] = b_new[index]/np.linalg.norm(A_new[index])
        A_new[index] = A_new[index]/np.linalg.norm(A_new[index])
        
    # Here, we scale b so that we can acurately tell if we passed or failed the constraint
    # It must be that the largest magnitude entry in b is at most 1, since we test constraints by multiply two unit vectors
    # This is a consequence of the Cauchey-Schwartz inequallity
    k = np.linalg.norm(b_new)
    b_new = b_new/k
    
    return A_new, b_new,k

In [128]:
A, b, k = conditionMatrixVector(A_prime, b_prime)

### Violated Constraints

In [170]:
# Here will be a function that generates a unitary gate given a unit vector row called a_i
# The point of this gate is that when given |0>, it will produce the state described by a_i

# The unitaries and controled versions which encode the rows of A
def create_A_gates(A):
    A_gates = {}
    A_c_gates = {}

    for index in range(len(A)):
        new_unitary = create_unitary(trans(A[index]))

        A_gates[index] = UnitaryGate(new_unitary,label=f'a_{index}')
        A_c_gates[index] = A_gates[index].control(1)
        
    return A_gates, A_c_gates

In [189]:
# Takes the inner product of a and x, and checks it against b to see if there has been a row violation

def InnerProduct(index,x):
    # Here we have made the controled x-unitary
    x_c_unitary = UnitaryGate(create_unitary(x),label='x_gate').control(1)
    
    dim = x.size
    num_qubits = int(2*math.log(dim,2)+1)
    
    qc = QuantumCircuit(num_qubits,1)
    
    qc.h(0)
    
    # Encode x on |0>
    qc.barrier()
    qc.x(0)
    # Here I append the unitary gates manually, need to fix
    qc.append(x_c_unitary, [0,1,2])
    qc.x(0)
    qc.barrier()
    
    # Encode the row on |1>
    # Here I append the unitary gates manually, need to fix
    qc.append(A_c_gates[index], [0,3,4])
    
    qc.h(0)
    
    qc.measure(0,0)
    
    # This section runs the quantum simulation
    job = transpile(qc,backend=simulator)
    shots = 1000000

    result = simulator.run(job, shots=shots).result()

    try:
        p = result.get_counts(qc)['0']/shots
    except:
        p = 0

    InnerProduct = 2*p - 1

    
    return InnerProduct

In [190]:
def NewCenter(A,b,center):
    violated_rows = []

    for index in range(len(A)):
        if InnerProduct(index,center) < b[index]:
            violated_rows.append(index)

    move_vector = np.zeros((dim_new, 1))

    for index in violated_rows:
        move_vector = move_vector + trans(A[index])

    center = center+move_vector
    center = center/np.linalg.norm(center)
    
    return center

In [191]:
# Definines a space such that $-x >= -3, y >= 4, x+y >= 5$

# This is the constraint matrix
A_prime = np.array([[-1,0],
                    [0,1],
                    [1.0,1.0]])

# This is the constraint vector
b_prime = np.array([[-3.0],
                    [4.0],
                    [5.0]])

dim_new = 2**math.ceil(math.log(len(A_prime[0]),2)+1)
dim_dif = dim_new - len(A_prime[0])

# Initial center point creation
center = []
for zero in range(dim_new-1):
    center.append([0])
    
center.append([1])
center = np.array(center)

# Generate needed unitaries
A_gates, A_c_gates = create_A_gates(A)

In [192]:
center = NewCenter(A,b,center)

In [184]:
print(A)
print('\n')
print(center*k)
print('\n')
print(b*k)

[[-1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]]


[[2.33914409]
 [5.64719337]
 [0.        ]
 [0.37096083]]


[[-3.        ]
 [ 4.        ]
 [ 3.53553391]]
